In [1]:
from game import *
import random
from torch import optim

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
device = 'cuda'
#device = 'cpu'


In [3]:
class AE(nn.Module):
    def __init__(self):
      super(AE, self).__init__()
      z_dim = 2
      self.conv_en1 = nn.Conv2d(2,128,3,padding=1)
      self.conv_en2 = nn.Conv2d(128,128,3,padding=1)
      self.conv_en3 = nn.Conv2d(128,128,3,padding=1)
      self.conv_en4 = nn.Conv2d(128,128,3,padding=1)
      self.conv_en5 = nn.Conv2d(128,128,3,padding=1)
      self.conv_en6 = nn.Conv2d(128,128,3,padding=1)
      self.conv_en7 = nn.Conv2d(128,128,3,padding=1)
      self.conv_en8 = nn.Conv2d(128,128,3,padding=1)
      self.conv_en9 = nn.Conv2d(128,128,3,padding=1)
    
      
      self.batch_en1 = nn.BatchNorm2d(128)
      self.batch_en2 = nn.BatchNorm2d(128)
      self.batch_en3 = nn.BatchNorm2d(128)
      self.batch_en4 = nn.BatchNorm2d(128)
      self.batch_en5 = nn.BatchNorm2d(128)
      self.batch_en6 = nn.BatchNorm2d(128)
      self.batch_en7 = nn.BatchNorm2d(128)
      self.batch_en8 = nn.BatchNorm2d(128)
      self.batch_en9 = nn.BatchNorm2d(128)
 
      self.dense_enc1 = nn.Linear(128*3*3, 1000)
      self.dense_enc2 = nn.Linear(1000, 1000)
      self.dense_encmean = nn.Linear(1000, z_dim)
      
      self.dense_dec1 = nn.Linear(z_dim, 1000)
      self.dense_dec2 = nn.Linear(1000, 128*3*3)
      
      self.conv_dec1 = nn.Conv2d(128,128,3,padding=1)
      self.conv_dec2 = nn.Conv2d(128,128,3,padding=1)
      self.conv_dec3 = nn.Conv2d(128,128,3,padding=1)
      self.conv_dec4 = nn.Conv2d(128,128,3,padding=1)
      self.conv_dec5 = nn.Conv2d(128,128,3,padding=1)
      self.conv_dec6 = nn.Conv2d(128,128,3,padding=1)
      self.conv_dec7 = nn.Conv2d(128,128,3,padding=1)
      self.conv_dec8 = nn.Conv2d(128,128,3,padding=1)
      self.conv_dec9 = nn.Conv2d(128,2,3,padding=1)
        
      self.batch_dec1 = nn.BatchNorm2d(128)
      self.batch_dec2 = nn.BatchNorm2d(128)
      self.batch_dec3 = nn.BatchNorm2d(128)
      self.batch_dec4 = nn.BatchNorm2d(128)
      self.batch_dec5 = nn.BatchNorm2d(128)
      self.batch_dec6 = nn.BatchNorm2d(128)
      self.batch_dec7 = nn.BatchNorm2d(128)
      self.batch_dec8 = nn.BatchNorm2d(128)
      self.batch_dec9 = nn.BatchNorm2d(2)

    
    def _encoder(self, x):
      h1 = F.relu(self.batch_en1(self.conv_en1(x)))
    
      h2 = F.relu(self.batch_en2(self.conv_en2(h1)))
      h3 = F.relu(self.batch_en3(self.conv_en3(h2))+h1)
      
      h4 = F.relu(self.batch_en4(self.conv_en4(h3)))
      h5 = F.relu(self.batch_en5(self.conv_en5(h4))+h3)
      
      h6 = F.relu(self.batch_en6(self.conv_en6(h5)))
      h7 = F.relu(self.batch_en7(self.conv_en7(h6))+h5)
    
      h8 = F.relu(self.batch_en8(self.conv_en8(h7)))
      h9 = F.relu(self.batch_en9(self.conv_en9(h8))+h5)
    
        
      x = h9.reshape(-1,128*3*3)
      x = F.relu(self.dense_enc1(x))
      x = F.relu(self.dense_enc2(x))
      mean = self.dense_encmean(x)

      return mean
    
 
    def _decoder(self, z):
      x = F.relu(self.dense_dec1(z))
      x = F.relu(self.dense_dec2(x))
      x = x.reshape(-1,128,3,3)
    
      h1 = F.relu(self.batch_dec1(self.conv_dec1(x)))
      h2 = F.relu(self.batch_dec2(self.conv_dec2(h1))+x)
    
      h3 = F.relu(self.batch_dec3(self.conv_dec3(h2)))  
      h4 = F.relu(self.batch_dec4(self.conv_dec4(h3))+h2)
    
      h5 = F.relu(self.batch_dec5(self.conv_dec5(h4)))
      h6 = F.relu(self.batch_dec6(self.conv_dec6(h5))+h4)

      h7 = F.relu(self.batch_dec7(self.conv_dec7(h6)))
      h8 = F.relu(self.batch_dec8(self.conv_dec8(h7))+h7)
    
      h9 = (self.batch_dec9(self.conv_dec9(h8)))
  
      x = self.batch_dec9(h9)
      x = F.sigmoid(x)
      return x

    def forward(self, x):
      z = self._encoder(x)
      x = self._decoder(z)
      return x
    
    def loss(self, x):
      z = self._encoder(x)
      x2 = self._decoder(z)
      loss_f = nn.MSELoss()
      loss1 = loss_f(x,x2)
      loss2 = torch.abs(z.sum(1).sum(0))
      #print("loss")
      #print(loss1)
      #print(loss2)
      return loss1 + (0.001 * loss2)

In [4]:
state_list = []
for i in range(1000):
    state = State()
    while True:
        if state.is_done():
            break
        state = state.next(random_action(state))
        state_list.append(state)

state_list2 = []

for i in range(100):
    state = State()
    while True:
        if state.is_done():
            break
        state = state.next(random_action(state))
        state_list2.append(state)


In [5]:
model = AE()
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.train()
BATCH_SIZE = 2
for epoch in range(100):
    losses = []
    random.shuffle(state_list)
    for index in range(0,len(state_list)-BATCH_SIZE,BATCH_SIZE):
        xs = [[state_list[index+i].pieces,state_list[index+i].enemy_pieces] for i in range(BATCH_SIZE)]
        xs = np.array(xs)
        xs = xs.reshape(-1,2,3,3)
        x = torch.tensor(xs,dtype=torch.float)
        x = x.to(device)
        #print(x)
        model.zero_grad()
        y = model(x)
        loss = model.loss(x)
        loss.backward()
        optimizer.step()
        losses.append(loss.cpu().detach().numpy())
    print("EPOCH: {} loss: {}".format(epoch, np.average(losses)))

/home/tugajin/.local/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


EPOCH: 0 loss: 0.1939614862203598
EPOCH: 1 loss: 0.17366982996463776
EPOCH: 2 loss: 0.1620606929063797
EPOCH: 3 loss: 0.13908545672893524
EPOCH: 4 loss: 0.12116961181163788
EPOCH: 5 loss: 0.11122557520866394
EPOCH: 6 loss: 0.10459206998348236
EPOCH: 7 loss: 0.10027719289064407
EPOCH: 8 loss: 0.096400685608387
EPOCH: 9 loss: 0.09373632818460464
EPOCH: 10 loss: 0.09107808768749237
EPOCH: 11 loss: 0.08890654146671295
EPOCH: 12 loss: 0.086377814412117
EPOCH: 13 loss: 0.084731325507164
EPOCH: 14 loss: 0.08278200775384903
EPOCH: 15 loss: 0.08128229528665543
EPOCH: 16 loss: 0.07939713448286057
EPOCH: 17 loss: 0.07786444574594498
EPOCH: 18 loss: 0.0766044333577156
EPOCH: 19 loss: 0.0751204639673233
EPOCH: 20 loss: 0.07434973865747452
EPOCH: 21 loss: 0.07296691834926605
EPOCH: 22 loss: 0.07151372730731964
EPOCH: 23 loss: 0.0711597427725792
EPOCH: 24 loss: 0.07022002339363098
EPOCH: 25 loss: 0.06936541199684143
EPOCH: 26 loss: 0.06888920813798904
EPOCH: 27 loss: 0.06811407953500748
EPOCH: 28 los

In [6]:
print(len(state_list))
print(len(state_list2))

7609
782


In [7]:
model.eval()
BATCH_SIZE = 1
for index in range(0,len(state_list2)-BATCH_SIZE,BATCH_SIZE):
    print(index)
    xs = [[state_list2[index+i].pieces,state_list2[index+i].enemy_pieces] for i in range(BATCH_SIZE)]
    xs = np.array(xs)
    xs = xs.reshape(-1,2,3,3)
    x = torch.tensor(xs,dtype=torch.float)
    x = x.to(device)
    print(x)
    #model.no_grad()
    with torch.no_grad():
        y = model(x)
        z = model._encoder(x)
    print(y)
    print(z)
    

0
tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 1.],
          [0., 0., 0.],
          [0., 0., 0.]]]], device='cuda:0')
tensor([[[[4.7567e-02, 4.1388e-02, 5.4077e-06],
          [5.4257e-02, 3.1126e-02, 4.5944e-02],
          [7.1841e-02, 5.4836e-02, 5.1540e-02]],

         [[3.8640e-02, 2.2263e-03, 9.9932e-01],
          [8.4199e-03, 1.0102e-02, 8.1570e-03],
          [3.6835e-02, 2.2715e-02, 1.2895e-02]]]], device='cuda:0')
tensor([[-0.1460,  1.0462]], device='cuda:0')
1
tensor([[[[0., 0., 1.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 1., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]], device='cuda:0')
tensor([[[[9.4540e-03, 6.0556e-05, 3.8646e-01],
          [6.3455e-02, 9.9756e-03, 1.0055e-01],
          [5.9845e-02, 2.4063e-02, 5.8361e-02]],

         [[3.0473e-02, 9.9645e-01, 1.8075e-03],
          [1.3289e-02, 3.7594e-03, 2.4423e-02],
          [6.7662e-02, 9.2696e-03, 8.7681e-02]]]], device='cuda:0'